#  Transforming Collections Data to Linked Art 
## National Gallery of Art

## Import What We Need for Notebook
See the [Transform Introduction](01-00-Transform-Intro.ipynb) for a description of the Python libraries used

In [47]:
try:
    import json
except:
    !pip install json
    import json 
    
try:
    import requests
except:
    !pip install requests
    import requests
    
try:
    import cromulent
except:
    !pip install cromulent
    import cromulent
    
import csv

from cromulent.model import factory
    
from lib import linkedart as la

try:
    import pandas as pd
except:
    !pip install pandas
    import pandas as pd
    

#  baseURI for JSON-LD document
baseURI = "https://www.nga.gov/collection/"


## Input Data
The input data file is from https://github.com/NationalGalleryOfArt/opendata/tree/main/data

In [48]:
file = './data/nga/input/objects.csv'

mpg = pd.read_csv(file,low_memory=False)
mpg.head()

,objectid,accessioned,accessionnum,locationid,title,displaydate,beginyear,endyear,visualbrowsertimespan,medium,...,visualbrowserclassification,parentid,isvirtual,departmentabbr,portfolio,series,volume,watermarks,lastdetectedmodification,customprinturl
0,113260,1,2000.127.20.1-193,NaN,"Lithographs, Volume 9",NaN,1804.0,1866.0,1801 to 1825,book of lithographs,...,volume,NaN,0,CG-E,NaN,NaN,NaN,NaN,2020-05-06 22:01:32.06-04,NaN
1,113833,1,2000.127.3.1-172,NaN,"Lithographs, Volume 12",NaN,1804.0,1866.0,1801 to 1825,book of lithographs,...,volume,NaN,0,CG-E,NaN,NaN,NaN,NaN,2020-05-06 22:01:32.06-04,NaN
2,114640,1,2000.127.8.1-110,NaN,"Lithographs, Volume 17",NaN,1804.0,1866.0,1801 to 1825,book of lithographs,...,volume,NaN,0,CG-E,NaN,NaN,NaN,NaN,2020-05-06 22:01:32.06-04,NaN
3,114855,1,2000.127.10.1-28,NaN,"Lithographs, Volume 19",NaN,1804.0,1866.0,1801 to 1825,book of lithographs,...,volume,NaN,0,CG-E,NaN,NaN,NaN,NaN,2020-05-06 22:01:32.06-04,NaN
4,119191,1,2001.100.2.b,NaN,Studies of Lago Maggiore and and the Entrance ...,c. 1700,1700.0,1700.0,1651 to 1700,brown ink over graphite on laid paper,...,drawing,119190.0,0,CG-E,NaN,NaN,NaN,NaN,2019-10-28 22:01:34.883-04,NaN


In [50]:

#remove BOM
s = open(file, mode='r', encoding='utf-8-sig').read()
open(file, mode='w', encoding='utf-8').write(s)

allObjects = csv.DictReader(open(file, mode='r',encoding='utf-8'))

for obj in allObjects:
    print(json.dumps(obj,indent=2))
    break 

{
  "objectid": "113260",
  "accessioned": "1",
  "accessionnum": "2000.127.20.1-193",
  "locationid": "",
  "title": "Lithographs, Volume 9",
  "displaydate": "",
  "beginyear": "1804",
  "endyear": "1866",
  "visualbrowsertimespan": "1801 to 1825",
  "medium": "book of lithographs",
  "dimensions": "",
  "inscription": "",
  "markings": "",
  "attributioninverted": "Gavarni, Paul",
  "attribution": "Paul Gavarni",
  "creditline": "Ailsa Mellon Bruce Fund",
  "classification": "Volume",
  "subclassification": "",
  "visualbrowserclassification": "volume",
  "parentid": "",
  "isvirtual": "0",
  "departmentabbr": "CG-E",
  "portfolio": "",
  "series": "",
  "volume": "",
  "watermarks": "",
  "lastdetectedmodification": "2020-05-06 22:01:32.06-04",
  "customprinturl": ""
}


## Map fields

In [51]:
  
 mapp =  {
    "id":"objectid",
    "accession_number":"accessionnum",
    "accession_date": "",
    "classification" : "classification",
    "title": "title",
    "alt_title": "",
    "notes": "",
    "date_created":"displaydate",
    "date_created_earliest": "beginyear",
    "date_created_latest": "endyear",
    "created_period":"",
    "created_dynasty":"",
    "created_inscriptions":"",
    "created_notes": "",
    "creator":"attribution",
    "physical_medium": "medium",
    "physical_style": "",
    "physical_technique": "",
    "physical_description": "",
    "physical_dimensions": "dimensions",
    "created_provenance": "" ,
    "credit_line": "creditline",
    "collection" : "departmentabbr",
    "current_status" : "",
    "current_owner" : ""
}

In [ ]:
def createObjProp(obj,mapp,baseURI):
    objProp = {}
    csv_keys = list(obj.keys())
    for key in csv_keys:
        for prop in mapp:
            if key == mapp[prop]:
                if prop == "creator":
                    objProp[prop] = [{"id": baseURI +"creatorid/" + obj[mapp["id"]] ,"name": obj[key],"role":"Artist"}]
                else:
                    objProp[prop] = obj[key]
    objProp["current_owner"] = {"name":"National Gallery of Art",
                                "location":"Washington, D.C., United States",
                                "type": "http://vocab.getty.edu/aat/300312281" ,
                                "type_label": ""}
    return objProp   

In [67]:

for obj in allObjects:
    # create object property dictionary
    objProp = createObjProp(obj,mapp,baseURI)
    
    id = objProp["id"]
    object_uri = baseURI + id
    
    # create obj description
    objLA = la.createObjDesc(objProp,la.objTypes,object_uri)
    
    # write to file 
    text_file = open("./data/nga/output/json/all/" + objProp["id"] + ".json", "wt")
    n = text_file.write(factory.toString(objLA, compact=False))
    text_file.close()